#### Importing Libraries

In [1]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
from pprint import pprint
import os
import pandas as pd
from scipy.special import softmax
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt

c:\Users\prakh\anaconda3\envs\goemotions\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Tokenizer, Model and Pipeline Setup

In [2]:
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")

In [3]:
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

In [4]:
goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3
)

#### Inference

In [5]:
all_emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [6]:
filtered_emotions = ['happy', 'sad', 'disgusted', 'angry', 'fearful', 'neutral', 'surprised']

In [7]:
all_to_filtered = {
    'happy' : ['admiration', 'approval', 'caring', 'desire', 'gratitude', 'joy', 'love', 'optimism', 'pride'],
    'sad' : ['embarrassment', 'grief', 'remorse', 'sadness'],
    'disgusted' : ['disappointment' ,'disapproval', 'disgust'],
    'angry' : ['anger', 'annoyance'],
    'fearful' : ['fear', 'nervousness'],
    'surprised' : ['amusement', 'confusion', 'curiosity', 'excitement', 'surprise'], 
    'neutral' : ['realization', 'neutral', 'relief']
}

In [8]:
def process_line(line):
    emotion_prediction = goemotions(line)[0]
    labels, scores = emotion_prediction['labels'], emotion_prediction['scores']
    emotion_prediction = dict(zip(labels, scores))
    for emotion in all_emotions:
        if emotion not in emotion_prediction:
            emotion_prediction[emotion] = 0
    # Filter out emotions by clubbing them 
    filtered_prediction = {}
    for filtered_emotion in filtered_emotions:
        filtered_prediction[filtered_emotion] = sum([emotion_prediction[emotion] for emotion in all_to_filtered[filtered_emotion]])
    # Normalize filtered_prediction using softmax
    labels, scores = list(filtered_prediction.keys()), list(filtered_prediction.values())
    # scores = softmax(scores)
    emotion_prediction = dict(zip(labels, scores))
    return pd.Series(emotion_prediction)

In [9]:
def process_line(line):
    emotion_prediction = goemotions(line)[0]
    labels, scores = emotion_prediction['labels'], emotion_prediction['scores']
    emotion_prediction = dict(zip(labels, scores))
    for emotion in all_emotions:
        if emotion not in emotion_prediction:
            emotion_prediction[emotion] = 0
    return pd.Series(emotion_prediction)

In [24]:
goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.0
)
def process_line_filtered(line):
    emotion_prediction = goemotions(line)[0]
    labels, scores = emotion_prediction['labels'], emotion_prediction['scores']
    emotion_prediction = dict(zip(labels, scores))
    for emotion in all_emotions:
        if emotion not in emotion_prediction:
            emotion_prediction[emotion] = 0
    # Filter out emotions by clubbing them
    filter_emotions = {}
    for filtered_emotion in filtered_emotions:
        filter_emotions[filtered_emotion] = sum([emotion_prediction[emotion] for emotion in all_to_filtered[filtered_emotion]])
    # print(filter_emotions)
    return pd.Series(filtered_emotions)

In [21]:
text = "I am so happy today"
print(process_line_filtered(text))

{'happy': 0.9931343586649746, 'sad': 0.0012720050290226936, 'disgusted': 0.0020010421285405755, 'angry': 0.0016621244722045958, 'fearful': 0.0006890575459692627, 'neutral': 0.020537185366265476, 'surprised': 0.06899285491090268}
0        happy
1          sad
2    disgusted
3        angry
4      fearful
5      neutral
6    surprised
dtype: object


In [11]:
folder_path = './script_csv'
output_path = './script_csv_go_emotion'
for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv'):
        data = pd.read_csv(os.path.join(folder_path, file_name))
        data[all_emotions] = 0
        data[all_emotions] = data['line'].progress_apply(process_line)
        data.to_csv(os.path.join(output_path, file_name), index=False)

100%|██████████| 15/15 [27:06<00:00, 108.45s/it]


In [25]:
folder_path = './script_csv'
output_path = './script_csv_go_emotion_filtered'
for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv'):
        data = pd.read_csv(os.path.join(folder_path, file_name))
        data[filtered_emotions] = 0
        data[filtered_emotions] = data['line'].progress_apply(process_line_filtered)
        data.to_csv(os.path.join(output_path, file_name), index=False)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [37:31<00:00, 150.13s/it]
